In [1]:

import zipfile
import io
import pandas as pd
import re
import numpy as np

# Reporte Q1 ETL

In [2]:
# Ruta del archivo ZIP
zip_path = r"C:\Users\aleja\Downloads\ReporteQ1.zip"

# Leer el contenido del archivo ZIP
with zipfile.ZipFile(zip_path, 'r') as zip_file:
    
    html_file_name = zip_path.split('\\')[-1].replace('.zip', '.html')
    
    # Leer el contenido del archivo HTML dentro del ZIP
    with zip_file.open(html_file_name) as html_file:
        # Leer todas las tablas del archivo HTML
        dfs = pd.read_html(io.BytesIO(html_file.read()))

# Usar los nombres de las columnas de la primera tabla
df_columns = dfs[0][:1].values

# Procesar cada tabla y unirlas
processed_dfs = []
for df in dfs:
    # Eliminar la primera y la última fila
    df = df.iloc[1:-1]
    # Resetear el índice
    df = df.reset_index(drop=True)
    processed_dfs.append(df)

# Unir todos los DataFrames procesados
df_q1 = pd.concat(processed_dfs, ignore_index=True)

# Asignar los nombres de columnas al DataFrame combinado
df_q1.columns = df_columns[0]

# Elimina la ultima fila de totales
df_q1 = df_q1[df_q1['Identificacion']!='Total']

# Lista de columnas que quieres convertir a número
numeric_columns = [col for col in df_q1.columns if col != 'Nombre']

# Función para limpiar y convertir a número
def limiar_y_convertir(x):
    if pd.isna(x):
        return x
    # Eliminar puntos y reemplazar comas por puntos
    cleaned = re.sub(r'[.,]', '', str(x))
    try:
        return int(cleaned)
    except ValueError:
        return x

# Aplicar la limpieza y conversión a las columnas numéricas
for col in numeric_columns:
    df_q1[col] = df_q1[col].apply(limiar_y_convertir)


# Crea la columna Total_cartera
df_q1['Total_cartera']  = df_q1['Total Mora'] + df_q1['Saldo Corriente']


In [7]:
#df_q1.to_excel('../files/dfq1.xlsx', sheet_name='base')

# Otras bases de datos

In [3]:
categorias = pd.read_excel('../datos/LISTADO_DE_ALIADOS_Y_CATEGORIAS.xlsx', sheet_name='CATEGORIAS QUERY')
recaudos = pd.read_csv('../datos/recaudos.csv', sep=';')
ventas_historicas = pd.read_csv('../datos/ventas_historicas.csv', sep=';')
reporte37 = pd.read_csv('../datos/reporte37.csv', sep=';')
datos_basicos = pd.read_csv('../datos/datosbasicos.csv', sep=';')

C:\Users\aleja\AppData\Local\Temp\ipykernel_8020\1489866834.py:5: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  datos_basicos = pd.read_csv('../datos/datosbasicos.csv', sep=';')


In [4]:

# Función para verificar si un valor es numérico 
def numerico(value):
    str(value).strip()
    try:
        int(value)
        return True
    except ValueError:
        return False
    

# Identificar las filas donde 'CEDULA' no es numérica
mask = ~datos_basicos['CEDULA'].apply(numerico)

cc_errada = datos_basicos[mask]

def limpiar_cedula(cedula):
    if pd.isna(cedula):
        return cedula
    # Eliminar todo excepto los dígitos
    cedula_limpia = re.sub(r'\D', '', cedula)
    # Si después de limpiar queda vacío, devolver NaN
    return pd.to_numeric(cedula_limpia) if cedula_limpia else 'error'

# Aplicar la función limpiar_cedula solo a las filas que necesitan ser limpiadas
datos_basicos.loc[mask, 'CEDULA'] = datos_basicos.loc[mask, 'CEDULA'].apply(limpiar_cedula)

# Cownvierte la columna CEDULA A int
datos_basicos['CEDULA'] = datos_basicos['CEDULA'].astype(float).astype('Int64')


# Elimina las CC duplicadas
datos_basicos.drop_duplicates(subset='CEDULA', inplace=True)

# Elimina las CC duplicadas
datos_basicos.drop_duplicates(subset='CEDULA', inplace=True)


# Merge datos basicos

In [5]:
merge_con_datosbasicos = pd.merge(df_q1, datos_basicos, left_on='Identificacion', right_on='CEDULA', how='left')

# Merge ventas

In [6]:
ventas_mes = pd.read_csv('../datos/ventas.csv', sep=';')

In [7]:
print("Duplicados en ventas_mes:")
print(ventas_mes.duplicated().sum())

print("\nDuplicados en ventas_historicas:")
print(ventas_historicas.duplicated().sum())

Duplicados en ventas_mes:
0

Duplicados en ventas_historicas:
1


In [8]:
nana = ventas_historicas[ventas_historicas.duplicated()]

In [9]:
nana.value_counts().sum()

np.int64(0)

In [10]:
merge_con_datosbasicos_ventas = pd.merge(merge_con_datosbasicos, ventas, left_on='Pagare', right_on='NUMERO_DE_CREDITO', how='left')

NameError: name 'ventas' is not defined

# Merge Categoria

In [ ]:
merge_con_datosbasicos_ventas_categ = pd.merge(merge_con_datosbasicos_ventas, categorias, on='TIENDA', how='left', suffixes=('', '_cat'))

In [11]:
# Genera archivo en csv
#merge_con_datosbasicos_ventas_categ.to_csv('../files/base_de_datos.csv', index=False)
merge_con_datosbasicos.to_excel('../files/base_de_datos.xlsx', index=False, sheet_name='Base de Datos')

: 